In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
# (6) run `make pipeline` to clean DB
links = pd.read_csv('data/export.csv')
links

/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/564014651.py:12: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99104,161764,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,\N,2023-01-24 12:59:30.014509+00,2023-01-24 12:59:30.014509+00,NaN,\N,CEA,37.0,0.0,\N
99105,161765,https://www.askamanager.org/2023/01/trainer-wa...,trainer wants us to “get emotional” with each ...,\N,\N,2023-01-24 12:59:30.015741+00,2023-01-24 12:59:30.015741+00,NaN,\N,AskManager,26.0,0.0,\N
99106,161766,https://warontherocks.com/2023/01/the-triumphs...,The Triumphs and Tribulations of Peter the Gre...,\N,\N,2023-01-24 12:59:30.016972+00,2023-01-24 12:59:30.016972+00,NaN,\N,WarOnRocks,96.0,0.0,\N
99107,161767,http://paulgraham.com/https://sep.yimg.com/ty/...,Chapter 2 of Ansi Common Lisp,\N,\N,2023-01-24 12:59:30.018258+00,2023-01-24 12:59:30.018258+00,NaN,\N,Graham,70.0,0.0,\N


In [2]:
links['added'].max()

'2023-01-24 12:59:30.019516+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299       1306
1300       1307
1301       1308
1302       1309
1303       1310
          ...  
98720    161363
98721    161364
98722    161365
98723    161367
99042    161700
Name: id, Length: 11829, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
99104,161764,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,\N,2023-01-24 12:59:30.014509+00,2023-01-24 12:59:30.014509+00,NaN,\N,CEA,37.0,0.0,\N


In [5]:
links[links['url'].duplicated()]['id']

99104    161764
Name: id, dtype: int64

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)


...1/169
...11/169


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...21/169
...31/169


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...

...141/169
...151/169


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...161/169
...169/169


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_86727/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99272,161932,https://www.wave.com/en/blog/early-execution/,A case study in early-stage startup execution ...,\N,\N,2023-01-24 11:56:23.628407,2023-01-24 11:56:23.628422,\N,\N,Custom,82.0,0.0,NaN
99273,161933,https://www.wave.com/en/blog/time-tracking/,How time tracking helped me be a better manage...,\N,\N,2023-01-24 11:56:23.640148,2023-01-24 11:56:23.640163,\N,\N,Custom,34.0,0.0,NaN
99274,161934,https://www.wired.co.uk/article/artificial-int...,The Dark Risk of Large Language Models WIRED UK,\N,\N,2023-01-24 11:56:23.650950,2023-01-24 11:56:23.650961,\N,\N,Custom,68.0,0.0,NaN
99275,161935,https://www.worksinprogress.co/issue/buyers-of...,Buyers of first resort - Works in Progress,\N,\N,2023-01-24 11:56:23.661659,2023-01-24 11:56:23.661679,\N,\N,Custom,7.0,0.0,NaN


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99272,161932,https://www.wave.com/en/blog/early-execution/,A case study in early-stage startup execution ...,\N,\N,2023-01-24 11:56:23.628407,2023-01-24 11:56:23.628422,\N,\N,Custom,82.0,0.0,NaN
99273,161933,https://www.wave.com/en/blog/time-tracking/,How time tracking helped me be a better manage...,\N,\N,2023-01-24 11:56:23.640148,2023-01-24 11:56:23.640163,\N,\N,Custom,34.0,0.0,NaN
99274,161934,https://www.wired.co.uk/article/artificial-int...,The Dark Risk of Large Language Models WIRED UK,\N,\N,2023-01-24 11:56:23.650950,2023-01-24 11:56:23.650961,\N,\N,Custom,68.0,0.0,NaN
99275,161935,https://www.worksinprogress.co/issue/buyers-of...,Buyers of first resort - Works in Progress,\N,\N,2023-01-24 11:56:23.661659,2023-01-24 11:56:23.661679,\N,\N,Custom,7.0,0.0,NaN


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99272,161932,https://www.wave.com/en/blog/early-execution/,A case study in early-stage startup execution ...,\N,\N,2023-01-24 11:56:23.628407,2023-01-24 11:56:23.628422,\N,\N,Custom,82,0,NaN
99273,161933,https://www.wave.com/en/blog/time-tracking/,How time tracking helped me be a better manage...,\N,\N,2023-01-24 11:56:23.640148,2023-01-24 11:56:23.640163,\N,\N,Custom,34,0,NaN
99274,161934,https://www.wired.co.uk/article/artificial-int...,The Dark Risk of Large Language Models WIRED UK,\N,\N,2023-01-24 11:56:23.650950,2023-01-24 11:56:23.650961,\N,\N,Custom,68,0,NaN
99275,161935,https://www.worksinprogress.co/issue/buyers-of...,Buyers of first resort - Works in Progress,\N,\N,2023-01-24 11:56:23.661659,2023-01-24 11:56:23.661679,\N,\N,Custom,7,0,NaN


In [10]:
links.to_csv('data/export.csv', index=False)